# Pesquisando por iniciativas/projetos com Dados Abertos Governamentais no Github

In [1]:
import requests
import pandas as pd
import time

TODO:

**Mapeamento das principais bibliotecas**

- [API da Câmara dos deputados](https://dadosabertos.camara.leg.br/swagger/api.html)

In [2]:
search_strings_libraries = []

Devido a estrutura de pesquisa no github as palavras chaves utilizadas para o 
news não foram tão eficiente, visto que preciso adicionar palavras de busca ou 
de forma conter uma das palavras ou conter todas as palavras na sequencia especificada.

Sendo assim:
- '"projeto" and "dados governo"',
- '"projeto" and "dados governamentais"',
- '"projeto" and "monitora" and "dados" and "governo"'

Não foram buscas efetivas, porém as palavras chaves inicias sim.

In [3]:
search_strings = [
            'dados abertos',
            'dados abertos brasil',
            'dados abertos governo',
            'dados abertos governamentais',
            'dados governamentais',
            'dados publicos abertos',
            'dados do governo',
            'analise de dados do governo',
            'analise de dados governamentais',
            'portal de dados do governo',
            'portal de dados governamentais',
            'portal publico do governo',
            'portal de dados abertos do governo',
        ]

'consumir' e 'publicação' não retornaram resultados e por isso foram retirados a fim de adicionar palavras que retornem resultados melhores
- 'consumir dados abertos do governo'
- 'consumir dados abertos governamentais'
- 'publicação de dados abertos do governo',
- 'publicação de dados governamentais',

In [4]:
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?q='
credentials=('lorenaps','ba6db863961f1221ef38f7294976f0dc8dc36731')

Verificando o limite de requisições

In [5]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1573144004},
 'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1573144004},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1573144004},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1573144004},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1573140464}}}

In [6]:
colunas=[
        'id',
        'full_name',
        'description',
        'owner_type', 
        'owner_api_url',
        'owner_url',
        'url',
        'api_url',
        'fork',
        'created_at',
        'updated_at',
        'size',
        'stargazers_count',
        'language',
        'has_issues',
        'has_wiki',
        'forks_count',
        'forks',
        'open_issues',
        'watchers',
        'timestamp_extract'
    ]


In [7]:
def add_resultado(item):
    
    df = pd.DataFrame([[
                        item.get('id'),
                        item.get('full_name', None),
                        item.get('description', None),      
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None),
                        str(time.time()).split('.')[0]]], columns=colunas)

    return df    

In [32]:
def extrair_dados(data, resultados):
    
    for item in data.get('items', None):
        
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)

    return resultados

Verificando limitação de extração de dados da API

In [11]:
mais_de_mil = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(mais_de_mil, auth=credentials)
t.json()

{'documentation_url': 'https://developer.github.com/v3/search/',
 'message': 'Only the first 1000 search results are available'}

In [39]:
resultados = pd.DataFrame(columns=['id',
                                    'full_name',
                                    'description',
                                    'owner_type', 
                                    'owner_api_url',
                                    'owner_url',
                                    'url',
                                    'api_url',
                                    'fork',
                                    'created_at',
                                    'updated_at',
                                    'size',
                                    'stargazers_count',
                                    'language',
                                    'has_issues',
                                    'has_wiki',
                                    'forks_count',
                                    'forks',
                                    'open_issues',
                                    'watchers',
                                    'timestamp_extract',
                                    'commits',
                                    'contributors',]) 

In [40]:
def percorrendo_paginas(url, resultados):
    
    print('\nPrimeira requisição')
    
    results = requests.get(url, auth=credentials)    
    data = dict(results.json())
    total = data.get('total_count', None)
        
    print(">>> Foram encontrados {0} resultados. Extraindo...".format(total))

    resultados = extrair_dados(data, resultados)
    
    iteracoes = total // 30 
    
    for iteracao in range(0, iteracoes):        
        header = dict(results.links)
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
            
            print("\nNext url: {0}".format(next_url))
            
            results = requests.get(next_url, auth=credentials)
            data = dict(results.json())
            resultados = extrair_dados(data, resultados)
        
    return resultados

In [41]:
for string in search_strings:

    print("\nExtraindo repositórios para a string: '{0}'".format(string))

    url = url_base + string + sort
    %%time
    resultados = percorrendo_paginas(url, resultados)


Extraindo repositórios para a string: 'dados abertos'
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs

Primeira requisição
>>> Foram encontrados 370 resultados. Extraindo...

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=2

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=3

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=4

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=5

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=6

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=7

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=8

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=9

Next url: https:

In [42]:
resultados.describe()

,id,full_name,description,owner_type,owner_api_url,owner_url,url,api_url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
count,570,570,537,570,570,570,570,570,570,570,...,421,570,570,570,570,570,570,570,0.0,0.0
unique,424,424,389,2,366,366,424,424,1,424,...,22,2,2,16,16,14,25,28,0.0,0.0
top,195294598,prodest/ckanext-data_es_theme,Plugin / Tema do Portal de Dados Abertos do Go...,User,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/prodest/ckanext-data_es_theme,https://api.github.com/repos/thenets/ckanext-d...,False,2019-07-04T19:52:41Z,...,Python,True,True,0,0,0,0,1573142179,NaN,NaN
freq,6,6,6,461,16,16,6,6,570,6,...,94,555,554,423,423,487,377,30,NaN,NaN


In [43]:
urls = resultados['url']
len(urls)

570

**Extraindo Commits e Contributors**

In [57]:
def extrair_commits(url_repo):
    
    contributors_url = url_repo + '/commits'
        
    contributors = requests.get(contributors_url, auth=credentials)
    contributors = len(contributors.json())
    
    return contributors

In [58]:
def extrair_contributors(url_repo):
    
    commits_url = url_repo + '/contributors'
    
    commits = requests.get(commits_url, auth=credentials)
    commits = len(commits.json())
    
    return commits

In [59]:
urls = resultados['api_url']

for url in urls:
        
    print('\n>>> ', url)
    
    commits = extrair_commits(url)
    contributors = extrair_contributors(url)

    print("Tem {0} Commits - {1} Contributors".format(commits,contributors))
    
    resultados.loc[resultados["api_url"] == url, 'commits'] = commits
    resultados.loc[resultados["api_url"] == url, 'contributors'] = contributors


>>>  https://api.github.com/repos/CamaraDosDeputados/dados-abertos
Tem 28 Commits - 4 Contributors

>>>  https://api.github.com/repos/dadosgovbr/catalogos-dados-brasil
Tem 30 Commits - 4 Contributors

>>>  https://api.github.com/repos/prefeiturasp/dados-educacao
Tem 18 Commits - 2 Contributors

>>>  https://api.github.com/repos/dadosgovbr/aplicativos-dados-brasil
Tem 30 Commits - 5 Contributors

>>>  https://api.github.com/repos/dadosgovbr/kit
Tem 30 Commits - 6 Contributors

>>>  https://api.github.com/repos/mapaslivres/localidades
Tem 30 Commits - 5 Contributors

>>>  https://api.github.com/repos/odufrn/odufrn-downloader
Tem 30 Commits - 6 Contributors

>>>  https://api.github.com/repos/brasilopen/brasilopen
Tem 10 Commits - 2 Contributors

>>>  https://api.github.com/repos/vitorbaptista/dados-abertos-camara.gov.br
Tem 30 Commits - 1 Contributors

>>>  https://api.github.com/repos/jonny-data/conheca-seu-vereador
Tem 30 Commits - 6 Contributors

>>>  https://api.github.com/repos/dado

Tem 30 Commits - 2 Contributors

>>>  https://api.github.com/repos/vgeorge/estados-brasileiros
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/paraibatransparente/dados
Tem 30 Commits - 2 Contributors

>>>  https://api.github.com/repos/CodeForCuritiba/Analises-Jupyter-Notebook
Tem 2 Commits - 1 Contributors

>>>  https://api.github.com/repos/grstavares/DadosAbertosTSE
Tem 5 Commits - 1 Contributors

>>>  https://api.github.com/repos/febr-team/febr-data
Tem 30 Commits - 1 Contributors

>>>  https://api.github.com/repos/brunousml/Maquiavel
Tem 7 Commits - 1 Contributors

>>>  https://api.github.com/repos/chris-redfield/open-up-with-simplicity
Tem 30 Commits - 3 Contributors

>>>  https://api.github.com/repos/fredbortolato/dados-abertos-alesp
Tem 15 Commits - 1 Contributors

>>>  https://api.github.com/repos/erickos/Case_BibliotecaUFRN_DadosAbertos
Tem 10 Commits - 1 Contributors

>>>  https://api.github.com/repos/Ermesoml/Dados-Abertos-Camara-Legislativa
Tem 30 Commits 

KeyboardInterrupt: 

In [44]:
resultados.head(5)

,id,full_name,description,owner_type,owner_api_url,owner_url,url,api_url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
0,29256552,CamaraDosDeputados/dados-abertos,Repositório do serviço de Dados Abertos da Câm...,Organization,https://api.github.com/users/CamaraDosDeputados,https://github.com/CamaraDosDeputados,https://github.com/CamaraDosDeputados/dados-ab...,https://api.github.com/repos/CamaraDosDeputado...,False,2015-01-14T17:32:49Z,...,None,True,True,7,7,203,135,1573142155,NaN,NaN
1,39256926,dadosgovbr/catalogos-dados-brasil,Mapeamento de iniciativas (e catálogos) de dad...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/catalogos-dados-...,https://api.github.com/repos/dadosgovbr/catalo...,False,2015-07-17T14:02:34Z,...,Python,True,True,34,34,0,106,1573142155,NaN,NaN
2,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,False,2016-11-10T13:35:40Z,...,Jupyter Notebook,True,True,18,18,1,48,1573142155,NaN,NaN
3,49720381,dadosgovbr/aplicativos-dados-brasil,Mapeamento de aplicativos e visualizações que ...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/aplicativos-dado...,https://api.github.com/repos/dadosgovbr/aplica...,False,2016-01-15T13:29:14Z,...,None,True,True,9,9,1,48,1573142155,NaN,NaN
4,20032900,dadosgovbr/kit,Kit de implementação do Plano de Dados Abertos,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/kit,https://api.github.com/repos/dadosgovbr/kit,False,2014-05-21T18:44:02Z,...,CSS,True,False,15,15,10,39,1573142155,NaN,NaN


In [45]:
file_path = '../data/repositories_v1_' + str(time.time()).split('.')[0] + '.csv'
file_path

'../data/repositories_v1_1573142291.csv'

In [46]:
resultados.to_csv(file_path, index=False)

### Extraindo contribuidores dos repositórios

In [ ]:
colunas_contributors = [ 'repo_id',
                          'repo_name',
                          'repo_url',
                          'repo_api_url',
                          'contributor_id',
                          'contributor_login',
                          'contributor_type'
                          'contributor_url',
                          'contributor_api_url',
                          'timestamp_extract']

In [ ]:
def get_contributors(url):

    list_contributors = []
    results = requests.get(url, auth=credentials)
    results = results.json()
    
    for result in results:
        contributor = {}
        
        contributor = {
            'contributor_id': result.get('id', None),
            'contributor_login': result.get('login', None),
            'contributor_type': result.get('type', None),
            'contributor_url': result.get('html_url', None),
            'contributor_api_url': result.get('url', None),
        }

        list_contributors.append(contributor)
        
    return list_contributors

In [ ]:
def add_contributor(repo, contributor):
            
    df = pd.DataFrame([[
                    repo.get('repo_id'),
                    repo.get('repo_name', None),
                    repo.get('repo_url', None),      
                    repo.get('repo_api_url', None),
                    contributor.get('contributor_id', None),
                    contributor.get('contributor_login', None),
                    contributor.get('contributor_type', None),
                    contributor.get('contributor_url', None),
                    contributor.get('contributor_api_url', None),
                    str(time.time()).split('.')[0]]], columns=colunas_contributors)

    return df  

In [ ]:
def save_contributors(contributors_df, repo, contributors):
    
    for contributor in contributors:
        
        contributors_df = pd.concat([contributors_df, 
                                     add_contributor(repo, contributor)], 
                                    ignore_index=True, 
                                    sort=False)
    return contributors_df

In [ ]:
def search_contributors(df):
    
    contributors_df = pd.DataFrame(columns = colunas_contributors)
        
    for repo in repos:

        repo = {
            'repo_id': df['id'],
            'repo_name': df['full_name'],
            'repo_url': df['url'],
            'repo_api_url': df['api_url']
        }
        
        url_contributors = repo.get('repo_api_url') + '/contributors'        
        contributors = get_contributors(url_contributors)
        
        contributors_df = save_contributors(contributors_df, repo, contributors)
        
        return contributors_df

In [ ]:
result_contributors = search_contributors(resultados)
result_contributors.describe()

In [ ]:
file_path = '../data/contributors_' + str(time.time()).split('.')[0] + '.csv'
file_path

In [ ]:
result_contributors.to_csv(file_path, index=False)